## Pouch Cell Model Parameter Identification

In this notebook, we present the single particle model with a two dimensional current collector. This is achieved via the potential-pair models introduced in Marquis et al. [[1]](https://doi.org/10.1149/1945-7111/abbce4) as implemented in PyBaMM. At a high-level this is accomplished as a potential-pair model which is resolved across the discretised spatial locations.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [1]:
import numpy as np
import pybamm

import pybop

go = pybop.plot.PlotlyManager().go
pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

## Generating Synthetic Data

To demonstrate parameter estimation, we first need some data. We will generate synthetic data using a forward model, which requires defining a parameter set and the model itself.

### Defining Parameters and Model

We start by creating an example parameter set and then instantiate the single-particle model (SPM):

In [2]:
parameter_set = pybamm.ParameterValues("Marquis2019")
parameter_set.update(
    {
        "Negative electrode active material volume fraction": 0.495,
        "Positive electrode active material volume fraction": 0.612,
    }
)
model = pybamm.lithium_ion.SPM(
    options={"current collector": "potential pair", "dimensionality": 2},
)

### Updating the Spatial Grid

Next, we update the number of spatial locations to solve the potential-pair model,

In [3]:
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 10,  # separator
    "x_p": 10,  # positive electrode
    "y": 5,     # y direction
    "z": 5,     # z direction
    "r_n": 10,  # negative particle radius
    "r_p": 10   # positive particle radius
}

## Simulating the Forward Model

We can then simulate the model using the `predict` method, with a default constant current to generate voltage data.

In [4]:
t_eval = np.arange(0, 900, 3)
sim = pybamm.Simulation(model, parameter_values=parameter_set, var_pts=var_pts)
sol = sim.solve(t_eval=t_eval, initial_soc=0.5)
voltage = sol["Voltage [V]"](t_eval)
current = sol["Current [A]"](t_eval)

/home/dibyendu/Oxford_Work/PyBOP/newenv/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



### Adding Noise to Voltage Data

To make the parameter estimation more realistic, we add Gaussian noise to the data.

In [5]:
sigma = 0.001
corrupt_values = voltage.data + np.random.normal(0, sigma, len(t_eval))

## Identifying the Parameters

We will now set up the parameter estimation process by defining the datasets for optimisation and selecting the model parameters we wish to estimate.

### Creating a Dataset

The dataset for optimisation is composed of time, current, and the noisy voltage data:

In [6]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_values,
    }
)

### Defining Parameters to Estimate

We select the parameters for estimation and set up their prior distributions and bounds:

In [7]:
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        prior=pybop.Gaussian(0.7, 0.05),
        bounds=[0.45, 0.9],
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        prior=pybop.Gaussian(0.58, 0.05),
        bounds=[0.5, 0.8],
    ),
]

For plotting purposes, we want additional variables to be stored in the problem class. These are defined as,

In [8]:
additional_variables = [
    "Negative current collector potential [V]",
    "Positive current collector potential [V]",
]

### Setting up the Optimisation Problem

With the datasets and parameters defined, we can set up the optimisation problem, its cost function, and the optimiser.

In [9]:
# problem = pybop.FittingProblem(
#     model, parameters, dataset, additional_variables=additional_variables
# )
# cost = pybop.SumSquaredError(problem)
# optim = pybop.CMAES(cost, max_iterations=30)


builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_set)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

options = pybop.PintsOptions(
    max_iterations=30,
)
optim = pybop.CMAES(problem, options=options)

### Running the Optimisation

We proceed to run the CMA-ES optimisation algorithm to estimate the parameters:

In [10]:
results = optim.run()

### Viewing the Estimated Parameters

After the optimisation, we can examine the estimated parameter values:

In [11]:
results.x

array([0.45002208, 0.5       ])

In [21]:
parameter_set.update(
    {
        "Negative electrode active material volume fraction": results.x[0],
        "Positive electrode active material volume fraction": results.x[1],
    }
)

In [23]:
sim = pybamm.Simulation(model, parameter_values=parameter_set, var_pts=var_pts)
sol = sim.solve(t_eval=t_eval, initial_soc=0.5)
# voltage = sol["Voltage [V]"](t_eval)
# current = sol["Current [A]"](t_eval)

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Spatial Plotting

We can now plot the spatial variables from the solution object. First, the final negative current collector potential can be displayed. In this example, this is just a reference variable, but could be used for fitting or optimisation in the correct workflows.

In [ ]:
#TODO: need fixing


sim = pybamm.Simulation(model, parameter_values=parameter_set, var_pts=var_pts)
sol = sim.solve(t_eval=t_eval, initial_soc=0.5)

# Get spatial points from the solution
y_nodes = sol["y [m]"].entries
z_nodes = sol["z [m]"].entries

go.Figure(
    [
        go.Contour(
            x=np.arange(0, y_nodes - 1, 1),
            y=np.arange(0, z_nodes - 1, 1),
            z=sol["Negative current collector potential [V]"].entries[-1],
            colorscale="Viridis",
        )
    ],
    layout=dict(
        title="Negative current collector potential [V]",
        xaxis_title="x node",
        yaxis_title="y node",
        width=600,
        height=600,
    ),
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

We can then plot the positive current collector potential,

In [ ]:
#TODO: need fixing

go.Figure(
    [
        go.Contour(
            x=np.arange(0, model.var_pts["y"] - 1, 1),
            y=np.arange(0, model.var_pts["z"] - 1, 1),
            z=sol["Positive current collector potential [V]"][:, :, -1],
            colorscale="Viridis",
        )
    ],
    layout=dict(
        title="Positive current collector potential [V]",
        xaxis_title="x node",
        yaxis_title="y node",
        width=600,
        height=600,
    ),
)

AttributeError: 'SPM' object has no attribute 'var_pts'

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [15]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:

In [16]:
pybop.plot.surface(optim);